In [1]:
import pandas as pd

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import round

In [2]:
context = SparkContext()

In [3]:
data = pd.read_csv("insurance.csv")

In [4]:
sqlcontext = SQLContext(context)

/Users/mh067948/miniconda3/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [5]:
insurance_df = sqlcontext.createDataFrame(data)

In [6]:
insurance_df.rdd.getNumPartitions()

12

In [7]:
insurance_df.show()

+---+------+------+--------+------+---------+------------------+
|age|   sex|   bmi|children|smoker|   region|           charges|
+---+------+------+--------+------+---------+------------------+
| 19|female|  27.9|       0|   yes|southwest|         16884.924|
| 18|  male| 33.77|       1|    no|southeast|         1725.5523|
| 28|  male|  33.0|       3|    no|southeast|          4449.462|
| 33|  male|22.705|       0|    no|northwest|       21984.47061|
| 32|  male| 28.88|       0|    no|northwest|         3866.8552|
| 31|female| 25.74|       0|    no|southeast|         3756.6216|
| 46|female| 33.44|       1|    no|southeast|         8240.5896|
| 37|female| 27.74|       3|    no|northwest|         7281.5056|
| 37|  male| 29.83|       2|    no|northeast|         6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.136919999997|
| 25|  male| 26.22|       0|    no|northeast|         2721.3208|
| 62|female| 26.29|       0|   yes|southeast|        27808.7251|
| 23|  male|  34.4|      

In [8]:
insurance_df.count()

1338

In [9]:
insurance_df.select('age', 'sex', 'smoker').show()

+---+------+------+
|age|   sex|smoker|
+---+------+------+
| 19|female|   yes|
| 18|  male|    no|
| 28|  male|    no|
| 33|  male|    no|
| 32|  male|    no|
| 31|female|    no|
| 46|female|    no|
| 37|female|    no|
| 37|  male|    no|
| 60|female|    no|
| 25|  male|    no|
| 62|female|   yes|
| 23|  male|    no|
| 56|female|    no|
| 27|  male|   yes|
| 19|  male|    no|
| 52|female|    no|
| 23|  male|    no|
| 56|  male|    no|
| 30|  male|   yes|
+---+------+------+
only showing top 20 rows



In [10]:
insurance_df.describe().show()

+-------+------------------+------+-----------------+------------------+------+---------+------------------+
|summary|               age|   sex|              bmi|          children|smoker|   region|           charges|
+-------+------------------+------+-----------------+------------------+------+---------+------------------+
|  count|              1338|  1338|             1338|              1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  null|30.66339686098655|   1.0949177877429|  null|     null|13270.422265141257|
| stddev|14.049960379216154|  null|6.098186911679014|1.2054927397819137|  null|     null|12110.011236694001|
|    min|                18|female|            15.96|                 0|    no|northeast|         1121.8739|
|    max|                64|  male|            53.13|                 5|   yes|southwest|       63770.42801|
+-------+------------------+------+-----------------+------------------+------+---------+------------------+



In [23]:
insurance_df.select('sex').distinct().show()

+------+
|   sex|
+------+
|female|
|  male|
+------+



In [24]:
insurance_df.crosstab('sex', 'smoker')\
            .select('sex_smoker','yes','no')\
            .show()

+----------+---+---+
|sex_smoker|yes| no|
+----------+---+---+
|      male|159|517|
|    female|115|547|
+----------+---+---+



In [26]:
insurance_df.select('age','sex','bmi','region','charges')\
            .where(insurance_df['age'] > 50)\
            .withColumnRenamed('sex','gender')\
            .limit(10)\
            .show()

+---+------+------+---------+------------------+
|age|gender|   bmi|   region|           charges|
+---+------+------+---------+------------------+
| 60|female| 25.84|northwest|28923.136919999997|
| 62|female| 26.29|southeast|        27808.7251|
| 56|female| 39.82|southeast|        11090.7178|
| 52|female| 30.78|northeast|        10797.3362|
| 56|  male|  40.3|southwest|         10602.385|
| 60|female|36.005|northeast|       13228.84695|
| 59|female| 27.72|southeast|        14001.1338|
| 63|female|23.085|northeast|       14451.83515|
| 55|female|32.775|northwest|       12268.63225|
| 63|  male| 28.31|northwest|        13770.0979|
+---+------+------+---------+------------------+



In [27]:
insurance_df.groupBy('sex')\
            .count()\
            .show()

+------+-----+
|   sex|count|
+------+-----+
|female|  662|
|  male|  676|
+------+-----+



In [49]:
gdc = insurance_df.groupBy('sex', 'smoker')\
            .count()\
            .withColumnRenamed('count','total')

prop = gdc\
        .withColumn('prop', round(gdc.total/insurance_df.count() * 100,2))\
        .drop('total')\
        .where(gdc['sex'] == 'female')\
        .show()

+------+------+-----+
|   sex|smoker| prop|
+------+------+-----+
|female|   yes| 8.59|
|female|    no|40.88|
+------+------+-----+



In [43]:
cbs = insurance_df.groupBy('smoker','sex')\
            .agg({'charges':'avg', 'smoker':'count'})\
            .withColumnRenamed('avg(charges)','avg_charges')\
            .withColumnRenamed('count(smoker)','smoker_cnt')\
            .show()

+------+------+-----------------+----------+
|smoker|   sex|      avg_charges|smoker_cnt|
+------+------+-----------------+----------+
|    no|female|8762.297299541135|       547|
|   yes|female|30678.99627626087|       115|
|   yes|  male|33042.00597528302|       159|
|    no|  male|8087.204731276594|       517|
+------+------+-----------------+----------+

